In [0]:
!apt-get update -qq 2>&1 > /dev/null
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 131323 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.1-0ubuntu3~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
%cd drive/My Drive/Project ABBYY/gicrapi

/content/drive/My Drive/Project ABBYY/gicrapi


In [4]:
!ls

2.ipynb   demo1.sh  parser.ipynb	       Untitled0.ipynb	vk_part.tnt.xml
demo1.py  gicr	    text_classification.ipynb  Untitled.ipynb


In [0]:
from lxml import etree
import numpy as np
import pandas as pd
import re

In [0]:
#Находит значение атрибута
def parse_attributes(attribute, line):
    result = re.search(r'{}=.(\w+).'.format(attribute), line)
    if result:
        return result.group(1)
    #Если информации нет, возвращает строку 'NA'
    return 'NA'

#Применяет ругулярку к строке
def parse_text(reg, line):
    result = re.search(reg, line)
    if result:
        return result.group(1)
    #<o composite="true"> не обробатывается (я не знаю что это) и еще что то непонятное
    #МОЖЕТ ВЕРНУТЬ ПУСТУЮ СТРОКУ
    return ""
        
def parse_article(file):
    df = pd.DataFrame()
    
    attributes_list = ["source", "genrei", "gender", "birth", "nickname",
                      "id", "loc", "month", "rule", "year"]
    
    attributes_dict = {}
    for attr in attributes_list:
        attributes_dict[attr] = []
    attributes_dict["text"] = []
    attributes_dict["words"] = []
    attributes_dict["infinitives"] = []
    attributes_dict["specification"] = []
    
    text = ""
    words = []
    infinitives = []
    specification = []
    
    for line in file:
        if line.startswith('</text>'):
            
            attributes_dict["text"].append(text)
            attributes_dict["words"].append(words)
            attributes_dict["infinitives"].append(infinitives)
            attributes_dict["specification"].append(specification)
            
            words = []
            specification = []
            infinitives = []
            text = ""
            
        elif line.startswith('<text'):
            
            for attr in attributes_list:
                if attr == "loc":
                    attributes_dict[attr].append(re.findall(r'loc=.(\w+).', line))
                else:
                    attributes_dict[attr].append(parse_attributes(attr, line))
        else:
            
            words.append(parse_text(r'(\S+)\s+\S+\s+\S+', line))
            infinitives.append(parse_text(r'\S+\s+\S+\s+(\S+)', line))
            specification.append(parse_text(r'\S+\s+(\S+)\s+\S+', line))
            text += line + "\n"
            
    df = df.from_dict(attributes_dict)
    return df
        

In [0]:
def parse_file():
    path_to_xml = "vk_part.tnt.xml"
    with open(path_to_xml) as file:
        return parse_article(file)

In [0]:
df = parse_file()


In [9]:
column = ['gender', 'infinitives', 'specification']
data = df[column]
data.dropna()

,gender,infinitives,specification
0,F,"[online, игрушка]","[-, Ncfpnn]"
1,F,"[мой, первый, сайт, !]","[P--msna, Momsn, Ncmsnn, SENT]"
2,F,"[мой, семья]","[P--fsna, Ncfsnn]"
3,F,"[кабо-верде, от, а, до, я, :, отдых, на, кабо-...","[Npnsan, Sp-l, C, Sp-g, P-1-snny, -, Ncmsnn, S..."
4,F,"[обзор, телевизор, samsung, 7-й, ,, 8-й, и, 9-...","[Ncmsnn, Ncmpgn, -, Momsnd, ,, Mofsld, C, Moms..."
5,F,"[йог, для, начинающий, :, исправляемый, осанка...","[Ncmsgy, Sp-g, Afpmpgf, -, Afpmsns, Ncfsan, -,..."
6,NA,"[женщина, в, ссора, мочь, говорить, любой, вещ...","[Ncfpny, Sp-l, Ncfsln, Vmip3p-a-e, Vmn----a-e,..."
7,NA,"[ахахахах, ), ), ), ), )]","[Npfpay, -, -, -, -, -]"
8,NA,"[я, думать, ,, что, один, из, самый, грустный,...","[P-1-snny, Vmip1s-a-e, ,, C, P--fsna, Sp-g, P-..."
9,NA,"[уникальный, явление, :, 4, январь, в, течение...","[Afpnsnf, Ncnsnn, -, Mc---d, Ncmsgn, Sp-a, Ncn..."


In [10]:
dt = data[data.gender != "NA"]
dt.shape

(2165, 3)

In [0]:
data_F = data[data.gender == "F"]
data_M = data[data.gender == "M"]

In [28]:
sh = list(range(data_F.shape[0]))
np.random.shuffle(sh)
index = sh[:data_M.shape[0]]
data_F = data_F.iloc[index]
dt = data_M.append(data_F, ignore_index=True)
dt

,gender,infinitives,specification
0,M,"[счастье, -, это, когда, быть, ,, кто, пожелат...","[Ncnsnn, -, P--nsnn, C, Vmip3s-a-e, ,, P--msdn..."
1,M,"[не, ссать, и, написать, свой, любимый, челове...","[Q, Vmm-2s-a-e, C, Vmif1s-a-p, P--msda, Afpmsd..."
2,M,"[такой, тема, :, кто, не, передасть, далі, -, ...","[P--fsna, Ncfsnn, -, P---snn, Q, R, Afpmsns, -..."
3,M,"[весь, лучший, момент, мой, лето, в, фотокнига...","[P---pna, Afpmpnf, Ncmpnn, P--nsga, Ncnsgn, Sp..."
4,M,"[уххй, ), так, и, спиздить, бы, :, d]","[Afpmpaf, -, P-----r, C, Vmis-sm--e, Q, -, -]"
5,M,"[я, нравиться, avicii/denis, rublev/natasha, b...","[P-1-sdn, Vmip3s-m-e, -, -, -, -, -, -, -, -, ..."
6,M,"[я, нравиться, sander, van, doorn/firebeatz/ka...","[P-1-sdn, Vmip3s-m-e, -, -, -, -, -, -, Sp-l, ..."
7,M,"[я, нравиться, chainsmokers, -, #selfie, на, р...","[P-1-sdn, Vmip3s-m-e, -, -, -, Sp-l, Ncnsln, N..."
8,M,"[я, нравиться, new, world, sound, ,, timmy, tr...","[P-1-sdn, Vmip3s-m-e, -, -, -, ,, -, -, -, -, ..."
9,M,"[работать, проверять, 23, 10, 2014]","[Vmip3s-a-e, Vmis-sma-e, Mc---d, Mc---d, Mc---d]"


In [0]:
X_F = data_F['infinitives']
X_M = data_M['infinitives']

In [0]:
sentences_F = {}
sentences_M = {}
sentences = {}

for sentence in X_F:
  for word in sentence:
    word = word.strip('(){}[]&?<>/\:;,*^!@#+-=._%$0123457689')
    if word != "":
      if sentences_F.get(word) != None:
          sentences_F[word] = sentences_F[word] + 1
      else:
          sentences_F.update({word : 1})
          
      if sentences.get(word) != None:
          sentences[word] = sentences[word] + 1
      else:
          sentences.update({word : 1})
          
          
for sentence in X_M:
  for word in sentence:
    word = word.strip('(){}[]&?<>/\:;,*^!@#+-=._%$0123457689')
    if word != "":
      if sentences_M.get(word) != None:
          sentences_M[word] = sentences_M[word] + 1
      else:
          sentences_M.update({word : 1})
          
      if sentences.get(word) != None:
          sentences[word] = sentences[word] + 1
      else:
          sentences.update({word : 1})

In [0]:
n = 0.03
count_word = sum(sentences.values())
dic_normal = {key : sentences[key] / count_word for key in sentences.keys()}
keyness_F = {}
count_F = sum(sentences_F.values())
sentences_F_normal = {key : sentences_F[key] / count_F for key in sentences_F.keys()}
keyness_F = {key : (sentences_F_normal[key] + n)/(dic_normal[key] + n) for key in sentences_F_normal.keys()}
count_M = sum(sentences_M.values())
sentences_M_normal = {key : sentences_M[key] / count_M for key in sentences_M.keys()}
keyness_M = {key : (sentences_M_normal[key] + n)/(dic_normal[key] + n) for key in sentences_M_normal.keys()}

In [32]:
print(len(sentences))

9505


In [17]:
print(keyness_F)
print(keyness_M)

{'ты': 1.032515620041349, 'икать': 1.0004361892615556, 'из-за': 1.0044753852986115, 'синхронный': 1.0004361892615556, 'миоклонический': 1.0004361892615556, 'сокращение': 1.0004361892615556, 'диафрагма': 1.0004361892615556, 'и': 1.032450833390069, 'межреберный': 1.0004361892615556, 'мышца': 0.9997642414697808, 'а': 0.9929888941154402, 'не': 1.0132070946941873, 'потому': 1.0022635182866908, 'что': 1.0162362025954836, 'кто-то': 1.0007196424888793, 'вспоминать': 1.0028023400836865, 'идиот': 0.9997642414697808, 'опубликованый': 1.0102864243930139, 'фото': 0.9997082402098525, 'в': 0.9874676376504001, 'instagram': 1.0112442022701114, 'лицей': 1.0004361892615556, 'http://instagram.com/p/txcxu7w-lb': 1.0004361892615556, 'этот': 1.0071261436278298, 'четверка': 1.0004361892615556, 'присесть': 1.0008717925858681, 'простой': 0.9941810807044541, 'взорвать': 1.0004361892615556, 'весь': 1.0054211529068737, 'интернет': 0.9955226120742058, 'мурашки': 1.0013068111527887, 'по': 0.9846629009783014, 'кожа':

#Used vector

In [19]:
pos = 0
keywords = {}
for key in keyness_F.keys():
  if keywords.get(key) == None and keyness_F[key] > 1.001:
    keywords.update({key : pos})
    pos = pos + 1
    
for key in keyness_M.keys():
  if keywords.get(key) == None and keyness_M[key] > 1.001:
    keywords.update({key : pos})
    pos = pos + 1
dic_kw = [*keywords]
len(dic_kw)

4594

In [0]:
X_new = np.array([0] * (dt.shape[0]*len(dic_kw))).reshape(dt.shape[0], len(dic_kw))
X = dt['infinitives']

n = 0
for sentence in X:
  for word in sentence:
    if keywords.get(word) != None:
      X_new[n][keywords[word]] += 1
  n += 1

In [21]:
X_new.shape

(1442, 4594)

In [0]:
y = dt['gender']
#y=y.astype('int')

    
from sklearn.model_selection import train_test_split
(X_train, X_test, y_train, y_test) = train_test_split(X_new, y, test_size=0.2, 
                                     random_state=0)


In [23]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(class_weight='balanced')
model.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='warn', n_jobs=None, penalty='l2', random_state=None,
          solver='warn', tol=0.0001, verbose=0, warm_start=False)

In [24]:
from sklearn.metrics import classification_report
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           F       0.64      0.64      0.64       143
           M       0.65      0.65      0.65       146

   micro avg       0.64      0.64      0.64       289
   macro avg       0.64      0.64      0.64       289
weighted avg       0.64      0.64      0.64       289



In [34]:
from sklearn.metrics import accuracy_score
print (accuracy_score(y_test, y_pred))

0.643598615916955


#Used features

In [0]:
X_oh = pd.DataFrame(columns=['F', 'M'])
X = dt['infinitives']
F = 0.
M = 0.
n = 0.
for sentence in X:
  F = 0.
  M = 0.
  for word in sentence:
    if keyness_F.get(word) != None:
      a = keyness_F[word]
    else:
      a = 0.
    F = F + a
    if keyness_M.get(word) != None:
      b = keyness_M[word]
    else:
      b = 0.
    M = M + b
  X_oh = X_oh.append({'F': F, 'M': M}, ignore_index=True)
 

In [35]:
X_oh

,F,M
0,21.120207,20.814778
1,9.118083,10.820122
2,57.865325,76.233387
3,8.971716,9.043581
4,4.025516,5.962755
5,4.989274,13.038219
6,2.994433,10.040453
7,2.994433,6.017881
8,4.979592,12.064421
9,1.993256,2.010392


In [37]:
X_oh.shape

(1442, 2)

In [0]:
(X_train, X_test, y_train, y_test) = train_test_split(X_oh, y, test_size=0.3, 
                                     random_state=0)

In [39]:
model = LogisticRegression(class_weight='balanced')
model.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='warn', n_jobs=None, penalty='l2', random_state=None,
          solver='warn', tol=0.0001, verbose=0, warm_start=False)

In [40]:
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           F       0.97      0.91      0.94       223
           M       0.91      0.97      0.94       210

   micro avg       0.94      0.94      0.94       433
   macro avg       0.94      0.94      0.94       433
weighted avg       0.94      0.94      0.94       433



In [41]:
print (accuracy_score(y_test, y_pred))

0.9376443418013857
